In [21]:
import pandas as pd
import os
from sqlalchemy import create_engine, text

pd.options.display.max_rows = 20
engine=create_engine('postgresql://postgres:sa@localhost/postgres', max_overflow=20)

def get_table_ddl_from_csv(table_name, path_to_csv, file_encoding):
    df = pd.read_csv(path_to_csv, encoding=file_encoding)
    
    column_list = "id serial not null, "
    for i in range(len(df.columns)):
        is_date = "date" in df.columns[i].lower()
        if is_date:
            data_type = "date"
        else:
            data_type = "varchar"
        column_name = df.columns[i].lower()
        column_list = column_list + f"{column_name} {data_type},"

    primary_key = f"CONSTRAINT {table_name}_pkey PRIMARY KEY (id)"
    query = f"CREATE TABLE {table_name} ({column_list} {primary_key});"
    return query

def execute_query(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return pd.DataFrame(result.fetchall(), columns=result.keys())

def execute_nonquery(sql):
    engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))

def execute_scalar(sql):
    result = engine.connect().execution_options(isolation_level="AUTOCOMMIT").execute((text(sql)))
    return result.fetchall()[0][0]

def load_data(table_name, path_to_csv, file_encoding):
    df = pd.read_csv(path_to_csv, encoding=file_encoding)
    df.columns

    column_list = ""
    for i in range(len(df.columns)):
        column_list = column_list + f"{df.columns[i]},"
    column_list = column_list[:-1]

    os_path = os.path.abspath(path_to_csv)
    options = f"format csv, header true, delimiter ',', encoding '{file_encoding}'"
    execute_nonquery(f"COPY {table_name}({column_list}) FROM '{os_path}' WITH ({options});")

In [13]:
# with open("data/inventory.csv") as f:
# file_encoding = f.encoding

execute_nonquery(get_table_ddl_from_csv("inventories", "data/inventory.csv", "ISO-8859-1"))
load_data("inventories", "data/inventory.csv", "ISO-8859-1")

COPY inventories(Date,Zone_ID,Zone_name,Zone_desc,Station_ID,Station_order,Station_name,Station_address,Station_description,Station_latitude,Station_longitude,Station_type,Station_capacity,Station_bikes,Station_places,Station_picture,Station_bikes_state,Station_places_state,Station_closed,Station_cdo) FROM '/Users/hugonne/DS4A/Repo/ds4a-encicla/notebooks/data/inventory.csv' WITH (format csv, header true, delimiter ',', encoding 'ISO-8859-1');


In [22]:
print(execute_scalar("SELECT COUNT (*) FROM inventories"))

72192
